In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import re
import plotly.graph_objects as go
from scipy.stats import ttest_ind
from sklearn.metrics import auc, roc_curve, classification_report

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_json('C:/Sophia/School!!!/2023 Spring 1/DS take home/8.Song/song.json',convert_dates = ['user_sign_up_date','time_played'])
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 4000 non-null   object        
 1   user_id            4000 non-null   int64         
 2   user_state         4000 non-null   object        
 3   user_sign_up_date  4000 non-null   datetime64[ns]
 4   song_played        4000 non-null   object        
 5   time_played        4000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 187.6+ KB


In [4]:
data['user_id'].nunique()

196

### Q1: What are the top 3 and the bottom 3 states in terms of number of users?

In terms of non-repeat users:
- The top 3 states are New York, California and Texas.
- The bottom 3 states are Arizona, New Mexico, Connecticut, Idaho, Nebraska, Rhode Island, Iowa, Kansas and North Dakota.

In [5]:
data.groupby(['user_state']).nunique().sort_values(by='user_id', ascending=False)[:3]

,id,user_id,user_sign_up_date,song_played,time_played
user_state,,,,,
New York,469,23,10,73,469
California,425,21,12,71,425
Texas,230,15,10,65,230


In [6]:
data.groupby(['user_state']).nunique().sort_values(by='user_id', ascending=True)[:9]

,id,user_id,user_sign_up_date,song_played,time_played
user_state,,,,,
Arizona,22,1,1,17,22
New Mexico,17,1,1,11,17
Connecticut,16,1,1,11,16
Idaho,26,1,1,17,26
Nebraska,36,1,1,24,36
Rhode Island,27,1,1,21,27
Iowa,23,1,1,17,23
Kansas,8,1,1,7,8
North Dakota,26,1,1,16,26


In terms of users, no matter repeat or not:

- The top 3 states are New York, California and Texas.
- The bottom 3 states are Kansas, Connecticut, New Mexico, and Virginia.

In [7]:
data.groupby(['user_state']).count().sort_values(by='id', ascending=False)[:3]

,id,user_id,user_sign_up_date,song_played,time_played
user_state,,,,,
New York,469,469,469,469,469
California,425,425,425,425,425
Texas,230,230,230,230,230


In [8]:
data.groupby(['user_state']).count().sort_values(by='id', ascending=True)[:4]

,id,user_id,user_sign_up_date,song_played,time_played
user_state,,,,,
Kansas,8,8,8,8,8
Connecticut,16,16,16,16,16
New Mexico,17,17,17,17,17
Virginia,17,17,17,17,17


### Q2: What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically define user engagement. What the CEO cares about here is in which states users are using the product a lot/very little.

In [9]:
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


##### We will use the definition of 'non-repeating users' to do the following analysis.

##### Definition: Average songs played per user in each states
- The top 3 states are Nebraska, Alaska and Mississippi.
- The bottom 3 states are Kansas, Virginia, and Minnesota.

In [10]:
(data.groupby(['user_state']).count()['song_played']/data.groupby(['user_state']).nunique()['user_id']).round(2).sort_values(ascending=False)[:3]

user_state
Nebraska       36.00
Alaska         29.00
Mississippi    28.33
dtype: float64

In [11]:
(data.groupby(['user_state']).count()['song_played']/data.groupby(['user_state']).nunique()['user_id']).round(2).sort_values(ascending=True)[:3]

user_state
Kansas        8.0
Virginia      8.5
Minnesota    10.5
dtype: float64

### Q3: The CEO wants to send a gift to the first user who signed-up for each state. That is, the first user who signed-up from California, from Oregon, etc. Can you give him a list of those users?

In [12]:
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [13]:
# looking for the first sign up date for each state
earliest_state = data[['user_state','user_sign_up_date']].groupby('user_state').min()
earliest_state.head(5)

,user_sign_up_date
user_state,
Alabama,2015-05-01
Alaska,2015-05-12
Arizona,2015-05-12
Arkansas,2015-05-08
California,2015-05-04


In [14]:
earliest_list = pd.merge(earliest_state, data[['user_id','user_state','user_sign_up_date']], how = 'left', on=['user_state','user_sign_up_date'])
earliest_list = earliest_list.drop_duplicates()
earliest_list.sort_values(by='user_state', ascending=True)

,user_state,user_sign_up_date,user_id
0,Alabama,2015-05-01,5
21,Alaska,2015-05-12,106
47,Arizona,2015-05-12,105
69,Arkansas,2015-05-08,78
78,California,2015-05-04,39
79,California,2015-05-04,44
119,Colorado,2015-05-19,173
122,Colorado,2015-05-19,166
154,Connecticut,2015-05-16,127
170,Florida,2015-05-04,41


### Q4: Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model.

In [15]:
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


### Collaborative Filtering
It is a technique that can filter out items that a user might like on the basis of reactions by similar users.

In [16]:
data.groupby(['song_played', 'user_id'])['id'].count().unstack(fill_value=0)

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0,0,1,3,0,2,0,0,0,0,...,0,0,3,3,0,2,0,0,2,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YOUR MOTHER SHOULD KNOW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Yellow Submarine,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Yesterday,1,1,0,0,0,0,0,0,1,0,...,1,1,1,1,2,2,0,0,0,1


In [17]:
# Step 1: build the Song-User matrix
song_user = data.groupby(['song_played', 'user_id'])['id'].count().unstack(fill_value=0)
song_user = (song_user > 0).astype(int)

song_user

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0,0,1,1,0,1,0,0,0,0,...,0,0,1,1,0,1,0,0,1,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YOUR MOTHER SHOULD KNOW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Yellow Submarine,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Yesterday,1,1,0,0,0,0,0,0,1,0,...,1,1,1,1,1,1,0,0,0,1


In [18]:
from sklearn.preprocessing import normalize

In [19]:
# Step 2: build song-song similarity matrix
song_user_norm = normalize(song_user, axis=1)  # normalize the song-user matrix
# we have to normalize this matrix, otherwise, some heavy users songs will dominant similarity score  
similarity = np.dot(song_user_norm, song_user_norm.T)  # calculate the similarity matrix
similarity_df = pd.DataFrame(similarity, index=song_user.index, columns=song_user.index)

similarity_df.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,Baby You're A Rich Man,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1.000000,0.263117,0.138675,0.14825,0.131559,0.299572,0.294174,0.098058,0.228802,0.200160,...,0.525213,0.113228,0.585429,0.278503,0.539411,0.087706,0.328897,0.377141,0.163430,0.0
A Hard Day's Night,0.263117,1.000000,0.000000,0.00000,0.100000,0.146385,0.111803,0.000000,0.000000,0.091287,...,0.305788,0.129099,0.266996,0.000000,0.157027,0.000000,0.050000,0.215003,0.074536,0.0
A Saturday Club Xmas/Crimble Medley,0.138675,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.161165,0.000000,0.000000,0.182574,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
ANYTIME AT ALL,0.148250,0.000000,0.000000,1.00000,0.000000,0.164957,0.094491,0.125988,0.000000,0.000000,...,0.172292,0.000000,0.188044,0.097590,0.176950,0.000000,0.000000,0.103835,0.000000,0.0
Across The Universe,0.131559,0.100000,0.000000,0.00000,1.000000,0.097590,0.000000,0.000000,0.000000,0.000000,...,0.101929,0.000000,0.133498,0.000000,0.104685,0.000000,0.000000,0.061430,0.000000,0.0


In [20]:
# Step 3: find the top-k most similar songs
def find_topk(song, similarity, k=1):
    df = similarity.loc[song].sort_values(ascending=False)[1:k + 1].reset_index()
    df = df.rename(columns={'song_played': 'Song', song: 'Similarity'})
    
    return df

In [21]:
# Example: find the top 10 similar song for 'Eight Days A Week'
df = find_topk(song='Eight Days A Week', similarity=similarity_df, k=10)
df

,Song,Similarity
0,Come Together,0.417399
1,Hey Jude,0.398410
2,While My Guitar Gently Weeps,0.381503
3,Penny Lane,0.380693
4,A Day In The Life,0.375960
5,Let It Be,0.349749
6,Get Back,0.341918
7,Revolution,0.338261
8,I've Got A Feeling,0.335083
9,We Can Work It Out,0.332900


### Q5: How would you set up a test to check whether your model works well and is improving engagement?

We need to perform a A/B test (reference):

- Randomly split users into two groups, one Control group and one Experiment group
- Control group has no recommendation strategy
- Experiment group recommend the next song
- After running some time, perform a one-tailed t-test on 'Average songs played per user'
    - Null hypothesis: population 'Average songs played per user' is same in two groups
    - Alternative hypothesis: Experiment group's population 'Average songs played per user' is higher than control group's population